In [1]:
import sys
!{sys.executable} -m pip install xgboost
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import itertools

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
        
test = pd.read_csv('C:/Users/Acer/Desktop/Univ Ai/Test Data.csv')
train = pd.read_csv('C:/Users/Acer/Desktop/Univ Ai/Training Data.csv')


#Shape of dataframe
print(" Shape of training dataframe: ", train.shape)
print(" Shape of testing dataframe: ", test.shape)


print(train.shape)
print(test.shape)

 Shape of training dataframe:  (252000, 13)
 Shape of testing dataframe:  (28000, 12)
(252000, 13)
(28000, 12)


In [3]:
print("names of columns", list(train.columns))
class_train = train.groupby('risk_flag').count()['Id'].reset_index().sort_values(by='Id',ascending=False)
print(class_train)

names of columns ['Id', 'income', 'age', 'experience', 'married', 'house_ownership', 'car_ownership', 'profession', 'city', 'state', 'current_job_years', 'current_house_years', 'risk_flag']
   risk_flag      Id
0          0  221004
1          1   30996


In [4]:
correlations_data = train.corr()['risk_flag'].sort_values()
print(correlations_data, '\n')

experience            -0.034523
age                   -0.021809
current_job_years     -0.016942
current_house_years   -0.004375
income                -0.003091
Id                     0.032153
risk_flag              1.000000
Name: risk_flag, dtype: float64 



In [5]:
#test.income=test.income.apply(lambda x: np.log10(x))
#train.income=test.income.apply(lambda x: np.log10(x))
test['income_log'] = np.log(test['income'])
train['income_log'] = np.log(train['income'])

In [6]:
test.state=test.state.apply(lambda x:x.replace(" ", "_"))
test.city=test.city.apply(lambda x:x.replace(" ", "_"))
test.profession=test.profession.apply(lambda x:x.replace(" ","_"))

In [7]:
train['state'] = np.where(train['state'] == 'Uttar_Pradesh[5]', 'Uttar_Pradesh', train['state'])
test['state'] = np.where(test['state'] == 'Uttar_Pradesh[5]', 'Uttar_Pradesh', test['state'])

In [8]:
state_risk_percent = pd.DataFrame(train.groupby(['state'])['risk_flag'].sum()/train.groupby(['state'])['risk_flag'].count()).reset_index()

In [9]:
state_risk_percent.columns = ['state', 'state_risk_percent']

In [10]:
train = pd.merge(train, state_risk_percent, how = 'left', on = 'state')
test = pd.merge(test, state_risk_percent, how = 'left', on = 'state')

In [11]:
train['profession'].unique()

array(['Mechanical_engineer', 'Software_Developer', 'Technical_writer',
       'Civil_servant', 'Librarian', 'Economist', 'Flight_attendant',
       'Architect', 'Designer', 'Physician', 'Financial_Analyst',
       'Air_traffic_controller', 'Politician', 'Police_officer', 'Artist',
       'Surveyor', 'Design_Engineer', 'Chemical_engineer',
       'Hotel_Manager', 'Dentist', 'Comedian', 'Biomedical_Engineer',
       'Graphic_Designer', 'Computer_hardware_engineer',
       'Petroleum_Engineer', 'Secretary', 'Computer_operator',
       'Chartered_Accountant', 'Technician', 'Microbiologist',
       'Fashion_Designer', 'Aviator', 'Psychologist', 'Magistrate',
       'Lawyer', 'Firefighter', 'Engineer', 'Official', 'Analyst',
       'Geologist', 'Drafter', 'Statistician', 'Web_designer',
       'Consultant', 'Chef', 'Army_officer', 'Surgeon', 'Scientist',
       'Civil_engineer', 'Industrial_Engineer', 'Technology_specialist'],
      dtype=object)

In [12]:
train['profession'] = np.where(train['profession'] == 'Mechanical_engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Design_Engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Chemical_engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Biomedical_Engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Computer_hardware_engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Petroleum_Engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Industrial_Engineer','Engineer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Civil_engineer','Engineer', train['profession'])

test['profession'] = np.where(test['profession'] == 'Mechanical_engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Design_Engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Chemical_engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Biomedical_Engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Computer_hardware_engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Petroleum_Engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Industrial_Engineer','Engineer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Civil_engineer','Engineer', test['profession'])


train['profession'] = np.where(train['profession'] == 'Fashion_Designer','Designer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Graphic_Designer','Designer', train['profession'])
train['profession'] = np.where(train['profession'] == 'Web_designer','Designer', train['profession'])

test['profession'] = np.where(test['profession'] == 'Fashion_Designer','Designer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Graphic_Designer','Designer', test['profession'])
test['profession'] = np.where(test['profession'] == 'Web_designer','Designer', test['profession'])

train['profession'] = np.where(train['profession'] == 'Financial_Analyst','Analyst', train['profession'])
train['profession'] = np.where(train['profession'] == 'Consultant','Analyst', train['profession'])

test['profession'] = np.where(test['profession'] == 'Financial_Analyst','Analyst', test['profession'])
test['profession'] = np.where(test['profession'] == 'Consultant','Analyst', test['profession'])

In [13]:
profession_risk_percent =  pd.DataFrame(train.groupby(['profession'])['risk_flag'].sum()/train.groupby(['profession'])['risk_flag'].count()).reset_index()

In [14]:
profession_risk_percent.columns = ['profession', 'profession_risk_percent']

In [15]:
train = pd.merge(train, profession_risk_percent, how = 'left', on = 'profession')
test = pd.merge(test, profession_risk_percent, how = 'left', on = 'profession')

In [16]:
# test.state.value_counts()
train['profession'].unique()

array(['Engineer', 'Software_Developer', 'Technical_writer',
       'Civil_servant', 'Librarian', 'Economist', 'Flight_attendant',
       'Architect', 'Designer', 'Physician', 'Analyst',
       'Air_traffic_controller', 'Politician', 'Police_officer', 'Artist',
       'Surveyor', 'Hotel_Manager', 'Dentist', 'Comedian', 'Secretary',
       'Computer_operator', 'Chartered_Accountant', 'Technician',
       'Microbiologist', 'Aviator', 'Psychologist', 'Magistrate',
       'Lawyer', 'Firefighter', 'Official', 'Geologist', 'Drafter',
       'Statistician', 'Chef', 'Army_officer', 'Surgeon', 'Scientist',
       'Technology_specialist'], dtype=object)

In [17]:
test['state_risk_percent'].describe()

count    28000.000000
mean         0.122905
std          0.017576
min          0.046053
25%          0.113254
50%          0.118039
75%          0.130586
max          0.215548
Name: state_risk_percent, dtype: float64

In [18]:
test.loc[pd.isnull(test['state_risk_percent']), 'state']

Series([], Name: state, dtype: object)

In [19]:
train.drop('state', axis =1, inplace = True)
test.drop('state', axis =1, inplace = True)
train.drop('profession', axis =1, inplace = True)
test.drop('profession', axis =1, inplace = True)

In [20]:
train[np.isinf(train['state_risk_percent'])]

,Id,income,age,experience,married,house_ownership,car_ownership,city,current_job_years,current_house_years,risk_flag,income_log,state_risk_percent,profession_risk_percent


In [21]:
train.head()

,Id,income,age,experience,married,house_ownership,car_ownership,city,current_job_years,current_house_years,risk_flag,income_log,state_risk_percent,profession_risk_percent
0,1,1303835,23,3,single,rented,no,Rewa,3,13,0,14.080820,0.154369,0.113552
1,2,7574516,40,10,single,rented,no,Parbhani,9,13,0,15.840300,0.113254,0.148427
2,3,3991815,66,4,married,rented,no,Alappuzha,4,10,0,15.199757,0.167097,0.134167
3,4,6256451,41,2,single,rented,yes,Bhubaneswar,2,12,1,15.649124,0.142550,0.148427
4,5,5768871,47,11,single,rented,no,Tiruchirappalli[10],3,14,1,15.567987,0.103163,0.115794


In [22]:
city_risk_percent = pd.DataFrame(train.groupby(['city'])['risk_flag'].sum()/train.groupby(['city'])['risk_flag'].count()).reset_index()
city_risk_percent.columns = ['city', 'city_risk_percent']

In [23]:
train = pd.merge(train, city_risk_percent, how = 'left', on = 'city')
test = pd.merge(test, city_risk_percent, how = 'left', on = 'city')

In [24]:

test[np.isinf(test['city_risk_percent'])]
test['city_risk_percent'].isnull().sum()

0

In [25]:
# test.loc[pd.isnull(test['city_risk_percent']), 'city'].unique()

In [26]:
train['income'].unique()

array([1303835, 7574516, 3991815, ..., 2843572, 4522448, 9070230],
      dtype=int64)

In [27]:
train.drop('city', axis =1, inplace = True)
test.drop('city', axis =1, inplace = True)

In [28]:
train[np.isinf(train['city_risk_percent'])]

,Id,income,age,experience,married,house_ownership,car_ownership,current_job_years,current_house_years,risk_flag,income_log,state_risk_percent,profession_risk_percent,city_risk_percent


In [29]:
train['city_risk_percent'].isnull().sum()

0

In [30]:
test['city_risk_percent'].describe()

count    28000.000000
mean         0.122755
std          0.047434
min          0.026114
25%          0.089186
50%          0.119821
75%          0.146497
max          0.326194
Name: city_risk_percent, dtype: float64

In [31]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [32]:
train.head()

,Id,income,age,experience,current_job_years,current_house_years,risk_flag,income_log,state_risk_percent,profession_risk_percent,city_risk_percent,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,1,1303835,23,3,3,13,0,14.080820,0.154369,0.113552,0.121554,0,1,0,0,1,1,0
1,2,7574516,40,10,9,13,0,15.840300,0.113254,0.148427,0.122497,0,1,0,0,1,1,0
2,3,3991815,66,4,4,10,0,15.199757,0.167097,0.134167,0.130814,1,0,0,0,1,1,0
3,4,6256451,41,2,2,12,1,15.649124,0.142550,0.148427,0.326194,0,1,0,0,1,0,1
4,5,5768871,47,11,3,14,1,15.567987,0.103163,0.115794,0.121137,0,1,0,0,1,1,0


In [33]:
train['house_ownership_norent_noown'].unique()

array([0, 1], dtype=uint64)

In [34]:
train.drop(['married_single'],axis =1 ,inplace =True)
train.drop(['car_ownership_yes'],axis =1 ,inplace =True)
train.drop(['house_ownership_norent_noown'],axis =1, inplace =True)
train.drop(['income'],axis =1 ,inplace =True)
test.drop(['married_single'],axis =1 ,inplace =True)
test.drop(['car_ownership_yes'],axis =1 ,inplace =True)
test.drop(['house_ownership_norent_noown'],axis =1, inplace =True)
test.drop(['income'],axis =1 ,inplace =True)

In [35]:
train.head()

,Id,age,experience,current_job_years,current_house_years,risk_flag,income_log,state_risk_percent,profession_risk_percent,city_risk_percent,married_married,house_ownership_owned,house_ownership_rented,car_ownership_no
0,1,23,3,3,13,0,14.080820,0.154369,0.113552,0.121554,0,0,1,1
1,2,40,10,9,13,0,15.840300,0.113254,0.148427,0.122497,0,0,1,1
2,3,66,4,4,10,0,15.199757,0.167097,0.134167,0.130814,1,0,1,1
3,4,41,2,2,12,1,15.649124,0.142550,0.148427,0.326194,0,0,1,0
4,5,47,11,3,14,1,15.567987,0.103163,0.115794,0.121137,0,0,1,1


In [36]:
y = train[['risk_flag']]
X= train.loc[:, train.columns != 'risk_flag']
X.shape

(252000, 13)

In [37]:
X.columns

Index(['Id', 'age', 'experience', 'current_job_years', 'current_house_years',
       'income_log', 'state_risk_percent', 'profession_risk_percent',
       'city_risk_percent', 'married_married', 'house_ownership_owned',
       'house_ownership_rented', 'car_ownership_no'],
      dtype='object')

In [38]:
X.head()

,Id,age,experience,current_job_years,current_house_years,income_log,state_risk_percent,profession_risk_percent,city_risk_percent,married_married,house_ownership_owned,house_ownership_rented,car_ownership_no
0,1,23,3,3,13,14.080820,0.154369,0.113552,0.121554,0,0,1,1
1,2,40,10,9,13,15.840300,0.113254,0.148427,0.122497,0,0,1,1
2,3,66,4,4,10,15.199757,0.167097,0.134167,0.130814,1,0,1,1
3,4,41,2,2,12,15.649124,0.142550,0.148427,0.326194,0,0,1,0
4,5,47,11,3,14,15.567987,0.103163,0.115794,0.121137,0,0,1,1


In [39]:
X = pd.get_dummies(X)
X.head()

,Id,age,experience,current_job_years,current_house_years,income_log,state_risk_percent,profession_risk_percent,city_risk_percent,married_married,house_ownership_owned,house_ownership_rented,car_ownership_no
0,1,23,3,3,13,14.080820,0.154369,0.113552,0.121554,0,0,1,1
1,2,40,10,9,13,15.840300,0.113254,0.148427,0.122497,0,0,1,1
2,3,66,4,4,10,15.199757,0.167097,0.134167,0.130814,1,0,1,1
3,4,41,2,2,12,15.649124,0.142550,0.148427,0.326194,0,0,1,0
4,5,47,11,3,14,15.567987,0.103163,0.115794,0.121137,0,0,1,1


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
k_fold = KFold(n_splits=10, shuffle=True, random_state= 100)
#cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )

In [41]:
X_train.drop(['Id'], axis =1).columns

Index(['age', 'experience', 'current_job_years', 'current_house_years',
       'income_log', 'state_risk_percent', 'profession_risk_percent',
       'city_risk_percent', 'married_married', 'house_ownership_owned',
       'house_ownership_rented', 'car_ownership_no'],
      dtype='object')

In [42]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(random_state= 0,class_weight ='balanced')
dtree_model.fit(X_train.drop(['Id'], axis =1),y_train)
dtree_pred = dtree_model.predict(X_test.drop(['Id'], axis =1))
print(confusion_matrix(y_test, dtree_pred))

[[39270  4979]
 [ 1141  5010]]


In [43]:
from sklearn.metrics import classification_report
y_pred__train = dtree_model.predict(X_train.drop(['Id'], axis =1))
print(classification_report(y_train, y_pred__train))

             precision    recall  f1-score   support

          0       1.00      0.91      0.96    176755
          1       0.62      1.00      0.77     24845

avg / total       0.95      0.93      0.93    201600



In [44]:
from sklearn.metrics import classification_report
y_pred = dtree_model.predict(X_test.drop(['Id'], axis =1))
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      0.89      0.93     44249
          1       0.50      0.81      0.62      6151

avg / total       0.91      0.88      0.89     50400



In [45]:
from sklearn.metrics import roc_auc_score

In [46]:
print(roc_auc_score(y_train, y_pred__train))

0.9574118329717912


In [47]:
print(roc_auc_score(y_test, y_pred))

0.8509896950754943


In [48]:
test['risk_flag'] = dtree_model.predict(test.drop(['id'],axis = 1))

In [49]:
test['risk_flag'].value_counts()

0    22399
1     5601
Name: risk_flag, dtype: int64

In [355]:
submit = test.loc[:, ['id', 'risk_flag']]

# Save the submission dataframe
submit.to_csv('predictions', index = False)